In [1]:
import os

In [2]:
pwd%%

'f:\\Files\\DS&ML\\E2E-Credit-Fraud-Detection\\Exp'

In [3]:
os.chdir("../")

In [4]:
pwd%%

'f:\\Files\\DS&ML\\E2E-Credit-Fraud-Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_preprocess: Path
    test_preprocess: Path
    model_name: str
    target_column: str
    random_search_params: dict
    n_iter: int     
    cv_folds: int
    scoring: str 
    n_jobs: int

In [7]:
from project.constants import *
from project.utils.common import *

In [8]:
import os
import joblib   
import numpy as np
import mlflow
import mlflow.xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
import dagshub
from project import logger
from project.entity.config_entity import ModelTrainerConfig

class ConfigurationManager:
    def __init__(self,
                config_filepath=CONFIG_PATH,
                schema_filepath=SCHEMA_PATH,
                params_filepath=PARAMS_PATH):
        
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_training_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBClassifier
        schema = self.schema
        random_search_params = params.random_search
        cv_params = params.cross_validation

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_preprocess=config.train_preprocess,
            test_preprocess=config.test_preprocess,
            model_name=config.model_name,
            target_column=schema.target_column.name,
            random_search_params=random_search_params, 
            cv_folds=cv_params.cv_folds,            
            scoring=cv_params.scoring,             
            n_jobs=cv_params.n_jobs,
            n_iter=cv_params.n_iter          
        )
        
        return model_trainer_config 

In [ ]:
from sklearn.model_selection import StratifiedKFold
import optuna

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        dagshub.init(repo_owner="JavithNaseem-J", repo_name="E2E-Credit-Fraud-Detection")
        mlflow.set_tracking_uri("https://dagshub.com/JavithNaseem-J/E2E-Credit-Fraud-Detection.mlflow")
        mlflow.set_experiment("E2E-Credit-Fraud-Detection")

    def train(self):
        # Validate file paths
        if not os.path.exists(self.config.train_preprocess):
            logger.error(f"Train preprocessed file not found at: {self.config.train_preprocess}")
            raise FileNotFoundError("Train preprocessed file not found")
        if not os.path.exists(self.config.test_preprocess):
            logger.error(f"Test preprocessed file not found at: {self.config.test_preprocess}")
            raise FileNotFoundError("Test preprocessed file not found")

        # Load preprocessed data
        train_data = np.load(self.config.train_preprocess, allow_pickle=True)
        test_data = np.load(self.config.test_preprocess, allow_pickle=True)

        logger.info(f'Loaded train and test data')
        logger.info(f'Train data shape: {train_data.shape}')
        logger.info(f'Test data shape: {test_data.shape}')

        train_x = train_data[:, :-1]
        train_y = train_data[:, -1]
        test_x = test_data[:, :-1]
        test_y = test_data[:, -1]

        # Optuna Optimization
        mlflow.xgboost.autolog()
        with mlflow.start_run(run_name="Optuna_HPO") as parent_run:
            parent_run_id = parent_run.info.run_id
            mlflow.set_tag("run_type", "hyperparameter_tuning")
            mlflow.set_tag("model", "XGBClassifier")

            def get_search_space(trial):
                return {
                    "n_estimators": trial.suggest_int("n_estimators", 50, 500),
                    "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
                    "max_depth": trial.suggest_int("max_depth", 3, 10),
                    "min_child_weight": trial.suggest_int("min_child_weight", 1, 7),
                    "gamma": trial.suggest_float("gamma", 0.0, 0.3),
                    "subsample": trial.suggest_float("subsample", 0.7, 1.0),
                    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 1.0),
                    "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
                    "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
                }

            def objective(trial):
                with mlflow.start_run(run_name=f"Trial_{trial.number}", nested=True):
                    mlflow.set_tag("mlflow.parentRunId", parent_run_id)
                    mlflow.set_tag("trial_number", trial.number)

                    params = get_search_space(trial)

                    model = XGBClassifier(
                        objective='binary:logistic',
                        verbosity=0,
                        eval_metric='logloss',
                        use_label_encoder=False,
                        **params
                    )

                    cv = StratifiedKFold(n_splits=self.config.cv_folds, shuffle=True, random_state=42)
                    cv_scores = cross_val_score(model, train_x, train_y, scoring=self.config.scoring, cv=cv, n_jobs=self.config.n_jobs)

                    mean_score = cv_scores.mean()
                    std_score = cv_scores.std()

                    mlflow.log_params(params)
                    mlflow.log_metric("cv_mean_accuracy", mean_score)
                    mlflow.log_metric("cv_std_accuracy", std_score)

                    logger.info(f"Trial {trial.number}: cv_mean_accuracy={mean_score:.4f} (+/- {std_score:.4f}), params={params}")

                    return mean_score

            logger.info('>>>>>>>>>> Starting Optuna Study <<<<<<<<<')

            study = optuna.create_study(direction="maximize")
            study.optimize(objective, n_trials=self.config.n_iter)

            logger.info(f"Best trial found: {study.best_trial.params} with accuracy {study.best_trial.value:.4f}")

            # Retrain best model on full training data
            best_params = study.best_trial.params
            best_model = XGBClassifier(
                objective='binary:logistic',
                verbosity=0,
                eval_metric='logloss',
                use_label_encoder=False,
                **best_params
            )
            best_model.fit(train_x, train_y)

            mlflow.xgboost.log_model(
                xgb_model=best_model,
                artifact_path="xgboost_model",
                registered_model_name="XGBClassifier_CreditFraud_Optuna"
            )
            logger.info("Best model logged to MLflow")

            # Save the model locally
            model_path = os.path.join(self.config.root_dir, self.config.model_name)
            joblib.dump(best_model, model_path)
            logger.info(f'Best model saved locally at {model_path}')


f:\ProgramFiles\anaconda3\envs\credit-fraud\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_training_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
    
except FileNotFoundError as e:
    logger.error(f'File Not Found: {e}')
except KeyError as e:
    logger.error(f'Key Error: {e}')
except AttributeError as e:
    logger.error(f'Attribute Error: {e}')
except Exception as e:
    logger.error(f'Unexpected error: {e}')

[2025-04-25 15:59:29,088: INFO: common: yaml file: yaml file\config.yaml loaded successfully]
[2025-04-25 15:59:29,099: INFO: common: yaml file: yaml file\schema.yaml loaded successfully]
[2025-04-25 15:59:29,111: INFO: common: yaml file: yaml file\params.yaml loaded successfully]
[2025-04-25 15:59:29,111: INFO: common: created directory at: artifacts]
[2025-04-25 15:59:29,111: INFO: common: created directory at: artifacts/model_trainer]
[2025-04-25 15:59:29,510: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as JavithNaseem-J

[2025-04-25 15:59:29,527: INFO: helpers: Accessing as JavithNaseem-J]
[2025-04-25 15:59:30,023: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/JavithNaseem-J/E2E-Credit-Fraud-Detection "HTTP/1.1 200 OK"]
[2025-04-25 15:59:30,434: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "JavithNaseem-J/E2E-Credit-Fraud-Detection"

[2025-04-25 15:59:30,444: INFO: helpers: Initialized MLflow to track repo "JavithNaseem-J/E2E-Credit-Fraud-Detection"]


...

[2025-04-25 16:01:22,663: INFO: 3253257748: Best trial found: {'n_estimators': 378, 'learning_rate': 0.25229404420160845, 'max_depth': 10, 'min_child_weight': 2, 'gamma': 0.07690507353035689, 'subsample': 0.998873914960003, 'colsample_bytree': 0.9471917148943463, 'reg_alpha': 0.36705792151218763, 'reg_lambda': 0.47540186707119947} with accuracy 0.9452]


2025/04/25 16:01:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "f:\ProgramFiles\anaconda3\envs\credit-fraud\lib\site-packages\xgboost\core.py:158: UserWarning: [16:01:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
f:\ProgramFiles\anaconda3\envs\credit-fraud\lib\site-packages\xgboost\core.py:158: UserWarning: [16:01:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/04/25 16:01:56 WARNING mlflow.models.model: Model logged without a signature and

[2025-04-25 16:02:03,113: INFO: 3253257748: Best model logged to MLflow]
[2025-04-25 16:02:03,139: INFO: 3253257748: Best model saved locally at artifacts/model_trainer\model.joblib]


Created version '2' of model 'XGBClassifier_CreditFraud_Optuna'.


🏃 View run Optuna_HPO at: https://dagshub.com/JavithNaseem-J/E2E-Credit-Fraud-Detection.mlflow/#/experiments/0/runs/47d601f413a0451ba4932d0fd348ff7e
🧪 View experiment at: https://dagshub.com/JavithNaseem-J/E2E-Credit-Fraud-Detection.mlflow/#/experiments/0
